# Simulated PIK statistics

Here we inspect the accuracy and characteristics of the PIKs assigned,
leveraging our knowledge of ground truth from pseudopeople.

It wouldn't be possible to do the ground truth part with the real PVS, but
Layne, Wagner, and Rothhaas did something similar by redacting SSN from real records,
sending them through PVS without the SSN, and then using the true SSN
as ground truth.
The health care records they used are probably quite different from a CUF,
but they found a **very** good overall PIK accuracy (see cell below).

In [1]:
# Query planning is now on by default, but it has some rough edges.
# See https://github.com/dask/dask/issues/10995 for general discussion
# and https://github.com/dask/dask-expr/issues/1060 for the particular
# issue I ran into.
import dask
dask.config.set({"dataframe.query-planning": False})

In [2]:
import datetime, os

from vivarium_research_prl import distributed_compute, utils
from IPython.display import display

In [3]:
print(datetime.datetime.now())

2024-05-13 11:02:49.856938


In [4]:
# DO NOT EDIT if this notebook is not called ground_truth_accuracy.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
simulated_data_output_dir = 'output/generate_simulated_data'
case_study_output_dir = 'output'

# The "compute engine" is what we use on the Python side
# for our case-study-specific operations,
# as opposed to the Splink engine
compute_engine = 'pandas'
# Only matter if using a distributed compute engine
compute_engine_num_jobs = 3
compute_engine_cpus_per_job = 2
compute_engine_memory_per_job = "5GB"
queue = "long.q"
local_directory = f"/tmp/{os.environ['USER']}_dask"

In [5]:
# Parameters
data_to_use = "usa"
simulated_data_output_dir = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study/generate_simulated_data/"
case_study_output_dir = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study/person_linkage_case_study/"
compute_engine = "dask"
compute_engine_num_jobs = 50
compute_engine_memory_per_job = "40GB"
compute_engine_cpus_per_job = 2
local_directory = "/mnt/team/simulation_science/priv/users/zmbc/prl/person_linkage_case_study_tmp/dask_local_run/dask_local"


In [6]:
# Parameters for a USA run
# data_to_use = "usa"
# simulated_data_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/generate_simulated_data"
# case_study_output_dir = "/ihme/scratch/users/zmbc/person_linkage_case_study/person_linkage_case_study"

# compute_engine = 'dask'
# compute_engine_num_jobs = 50
# compute_engine_memory_per_job = "120GB"
# compute_engine_cpus_per_job = 2

In [7]:
case_study_output_dir = f'{case_study_output_dir}/{data_to_use}'
simulated_data_output_dir = f'{simulated_data_output_dir}/{data_to_use}'

In [8]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=compute_engine_num_jobs,
    cpus_per_job=compute_engine_cpus_per_job,
    memory_per_job=compute_engine_memory_per_job,
    queue=queue,
    local_directory=local_directory,
)

/mnt/share/homes/zmbc/mambaforge/envs/person_linkage_case_study_20240423/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45801 instead
  warnings.warn(


<Client: 'tcp://10.158.111.9:41223' processes=50 threads=50, memory=1.82 TiB>

In [9]:
census_2030_piked = df_ops.read_parquet(f'{case_study_output_dir}/census_2030_piked.parquet')
confirmed_piks_with_ground_truth = df_ops.read_parquet(f'{case_study_output_dir}/confirmed_piks.parquet')

In [10]:
piked_proportion = df_ops.compute(census_2030_piked.pik.notnull().mean())
# Compare with 90.28% of input records PIKed in the 2010 CUF,
# as reported in Wagner and Layne, Table 2, p. 18 
print(f'{piked_proportion:.2%} of the input records were PIKed')

88.62% of the input records were PIKed


In [11]:
# Multiple Census rows assigned the same PIK, indicating the model thinks they are duplicates in Census
pik_sizes = df_ops.persist(df_ops.groupby_agg_small_groups(census_2030_piked, by='pik', agg_func=lambda x: x.size()))
df_ops.compute(pik_sizes.value_counts())

1    302545510
2       684065
3          668
4            4
Name: count, dtype: int64

In [12]:
# Interesting: in pseudopeople, sometimes siblings are assigned the same (common) first name, making them almost identical.
# The only giveaway is their age and DOB.
# Presumably, this tends not to happen in real life.
duplicate_piks = pik_sizes.rename('pik_size').reset_index().pipe(lambda df: df[df.pik_size > 1])

df_ops.head(census_2030_piked.merge(duplicate_piks, on="pik").sort_values('pik'))

,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,record_id,pik,pik_size
2009,93_411256,Austin,L,Carroll,11,12/04/2018,3701,albemarle dri,<NA>,pleasant grove,AL,35654,Household,Other relative,Male,White,2030,simulated_census_2030_2_1025398,0_1000309,2
2705,93_59123,Austin,L,Carroll,11,12/04/2018,1252,w 72nd ave,<NA>,rochester,NY,10075,Household,Reference person,Male,White,2030,simulated_census_2030_2_124106,0_1000309,2
1413,6800_184521,Kinley,M,Murray,16,03/11/2014,12801,park plza,<NA>,st. peters,MO,63115,Household,Biological child,Female,White,2030,simulated_census_2030_233_388109,0_10005,2
3004,6800_487914,Kinley,M,Murray,16,03/11/2014,671,new hope rd,<NA>,englewood,CO,80916,Household,Other relative,Female,White,2030,simulated_census_2030_233_1027806,0_10005,2
1431,6790_298700,David,J,Barrick,68,10/24/1961,345,iefferson ave,<NA>,<NA>,MO,65441,Household,Opposite-sex spouse,Male,White,2030,simulated_census_2030_231_627034,0_1000919,2
3685,6790_298700,James,J,Barrick,36,10/24/1961,345,jefferson ave,<NA>,jefferson city,MO,65441,Household,Adopted child,Male,White,2030,simulated_census_2030_231_627035,0_1000919,2
3753,8527_376719,Jacob,J,Tapley,17,04/20/2012,1810,a st southeast,<NA>,fairborn,OH,45417,Household,Other nonrelative,Male,White,2030,simulated_census_2030_285_789162,0_1001605,2
3957,8527_571898,Jacob,J,Tapley,17,04/20/2012,314,hicks hill road,<NA>,florissant,MO,64068,Household,Other relative,Male,White,2030,simulated_census_2030_285_1025935,0_1001605,2
1000,1362_3,Emily,S,Elwell,21,12/17/2008,505,e gn st,<NA>,elk point,SD,57029,College,Noninstitutionalized group quarters population,Female,Black,2030,simulated_census_2030_46_47324,0_1001652,2
3748,1362_22687,Emily,S,Elwell,21,12/17/2008,127,tri st,<NA>,addison,IL,60555,Household,Other relative,Female,Black,2030,simulated_census_2030_46_1025428,0_1001652,2


## Ground truth statistics

In [13]:
census_2030_ground_truth = df_ops.persist(
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_census_2030_ground_truth.parquet')
)

In [14]:
# In this version of pseudopeople, there are no actual duplicates in Census,
# which means all of the duplicates identified above are wrong.
assert len(census_2030_ground_truth) == len(df_ops.drop_duplicates(census_2030_ground_truth))

In [15]:
reference_files_ground_truth = df_ops.persist(df_ops.concat([
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_geobase_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_name_dob_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
], ignore_index=True))

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    8.720000e+02
mean     1.215106e+08
std      4.555555e+07
min      7.575983e+07
25%      7.598293e+07
50%      1.214460e+08
75%      1.670462e+08
max      1.673512e+08
dtype: float64
Creating partitions of 2,119MB


In [16]:
# However, there can be reference file records that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(reference_files_ground_truth, by='record_id', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1    1229041214
2      59021232
3       1771663
4        102952
5          6057
6           355
7            28
9             1
Name: count, dtype: int64

In [17]:
reference_files_ground_truth = df_ops.persist(reference_files_ground_truth.merge(
    n_unique_simulants,
    on='record_id',
    how='left',
))
reference_files_ground_truth.head(n=100)

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_0_1007416,3202_98373,1
1,simulated_geobase_reference_file_0_1009259,2416_1130696,1
2,simulated_geobase_reference_file_0_101754,6539_783854,1
3,simulated_geobase_reference_file_0_1030795,9272_24332,1
4,simulated_geobase_reference_file_0_1059381,5698_1116162,1
...,...,...,...
95,simulated_geobase_reference_file_0_486819,1753_69067,1
96,simulated_geobase_reference_file_0_499302,5949_192171,2
97,simulated_geobase_reference_file_0_499302,4344_791440,2
98,simulated_geobase_reference_file_0_531999,2047_838586,1


In [18]:
df_ops.head(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == df_ops.compute(reference_files_ground_truth.n_unique_simulants.max())])

,record_id,simulant_id,n_unique_simulants
12778090,simulated_geobase_reference_file_369_1409656,6554_95393,9
12778091,simulated_geobase_reference_file_369_1409656,6554_95397,9
12778092,simulated_geobase_reference_file_369_1409656,6554_95392,9
12778093,simulated_geobase_reference_file_369_1409656,6554_95390,9
12778094,simulated_geobase_reference_file_369_1409656,6554_95396,9
12778095,simulated_geobase_reference_file_369_1409656,6554_95389,9
12778096,simulated_geobase_reference_file_369_1409656,6554_95403,9
12778097,simulated_geobase_reference_file_369_1409656,6554_95406,9
12778098,simulated_geobase_reference_file_369_1409656,6554_95395,9


In [19]:
census_2030_ground_truth = df_ops.persist(census_2030_ground_truth.merge(
    df_ops.drop_duplicates(reference_files_ground_truth[['simulant_id']]).assign(possible_to_pik=1),
    on='simulant_id',
    how='left',
).assign(possible_to_pik=lambda df: df.possible_to_pik.fillna(0)))
possible_to_pik_proportion = df_ops.compute(census_2030_ground_truth.possible_to_pik.mean())
print(
    f'{(1 - possible_to_pik_proportion):.2%} of the input records are '
    'impossible to PIK correctly, since they are not in any reference files'
)

0.43% of the input records are impossible to PIK correctly, since they are not in any reference files


In [20]:
print(
    f'Assigned PIKs to {(piked_proportion / possible_to_pik_proportion):.2%} of PIK-able records'
)

Assigned PIKs to 89.00% of PIK-able records


In [21]:
reference_file = df_ops.concat([
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_geobase_reference_file.parquet',
    ),
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_name_dob_reference_file.parquet',
    ),
], ignore_index=True)

Imbalanced dataframe: too_few=False, too_many=True, too_large=False
count    7.700000e+02
mean     3.880555e+08
std      1.573591e+08
min      2.302881e+08
25%      2.308000e+08
50%      3.859436e+08
75%      5.455854e+08
max      5.482611e+08
dtype: float64
Creating partitions of 5,976MB


In [22]:
reference_file_piks = df_ops.persist(reference_file[['record_id', 'pik']])
reference_file_piks

,record_id,pik
npartitions=54,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [23]:
assert len(reference_file_piks) == len(df_ops.drop_duplicates(reference_file_piks[['record_id']]))

In [24]:
pik_simulant_pairs = df_ops.persist(df_ops.drop_duplicates(reference_files_ground_truth.merge(reference_file_piks, on='record_id')[['pik', 'simulant_id']]))

In [25]:
# However, there can be PIKs that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(pik_simulant_pairs, by='pik', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1      353924439
2       41181290
3        3259303
4         225868
5          14701
         ...    
247            1
216            1
205            1
55             1
157            1
Name: count, Length: 334, dtype: int64

In [26]:
pik_simulant_pairs = df_ops.persist(pik_simulant_pairs.merge(
    n_unique_simulants,
    on='pik',
    how='left',
))
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=108,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [27]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == df_ops.compute(pik_simulant_pairs.n_unique_simulants.max())])

,pik,simulant_id,n_unique_simulants
3155742,44_705631,9526_1151086,334
3155743,44_705631,2787_14227,334
3155744,44_705631,3374_167624,334
3155745,44_705631,4950_1033583,334
3155746,44_705631,3713_1038605,334
3155747,44_705631,4203_64253,334
3155748,44_705631,3984_1095858,334
3155749,44_705631,8221_85308,334
3155750,44_705631,9723_1146759,334
3155751,44_705631,315_103299,334


## Definitions of accuracy

1. (most strict) Assigning any PIK with multiple simulants is incorrect
2. Assigning a PIK with multiple simulants is neither incorrect nor correct (excluded from denominator)
3. (most lenient) Assigning a PIK with multiple simulants is correct, as long as at least one of those simulants matches the truth

In [28]:
# All modules, Medicare database, calculated from Layne, Wagner, and Rothhaas Table 1 (p. 15)
real_life_pvs_accuracy = 1 - (2_585 + 60_709 + 129_480 + 89_094) / (52_406_981 + 5_170_924 + 49_374_794 + 50_327_034)
f'{real_life_pvs_accuracy:.5%}'

'99.82079%'

### Definition 1

In [29]:
piks_assigned = df_ops.compute(census_2030_piked.pik.notnull().sum())
piks_assigned

303915660

In [30]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants > 1])

,pik,simulant_id,n_unique_simulants
4,0_1000242,3528_680562,2
5,0_1000242,3528_234421,2
13,0_1001101,1219_1068536,2
14,0_1001101,1219_24209,2
18,0_1001592,9776_900062,2
19,0_1001592,7107_889225,2
28,0_1002408,5860_796821,2
29,0_1002408,5969_787903,2
30,0_1002504,1232_155899,2
31,0_1002504,1232_44116,2


In [31]:
single_sim_piks_correct = df_ops.compute(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_piks_correct

267617405

In [32]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / piks_assigned
)

0.8805647099593354

In [33]:
assert len(confirmed_piks_with_ground_truth) == piks_assigned

In [34]:
df_ops.head(census_2030_ground_truth.rename(columns={'record_id': 'record_id_census_2030'}))

,record_id_census_2030,simulant_id,possible_to_pik
0,simulated_census_2030_0_37,28_44,1.0
1,simulated_census_2030_0_311,28_365,1.0
2,simulated_census_2030_0_314,28_370,1.0
3,simulated_census_2030_0_349,28_414,1.0
4,simulated_census_2030_0_680,28_804,1.0
5,simulated_census_2030_0_738,28_867,1.0
6,simulated_census_2030_0_1129,28_1312,1.0
7,simulated_census_2030_0_1197,28_1388,1.0
8,simulated_census_2030_0_1626,28_1892,1.0
9,simulated_census_2030_0_1642,28_1912,1.0


In [35]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_correct = df_ops.compute(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_record_links_correct

284389384

In [36]:
(
    single_sim_record_links_correct / piks_assigned
)

0.9357510040779077

### Definition 2

In [37]:
single_sim_piks_assigned = len(census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == 1][['pik', 'simulant_id']]))
single_sim_piks_assigned

267814885

In [38]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / single_sim_piks_assigned
)

0.9992626250030875

In [39]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_assigned = df_ops.compute(
    (confirmed_piks_with_ground_truth
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .n_unique_simulants == 1).sum()
)
single_sim_record_links_assigned

284600979

In [40]:
(
    single_sim_record_links_correct / single_sim_record_links_assigned
)

0.9992565204773944

### Definition 3

In [41]:
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
npartitions=108,,,
,string,string,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [42]:
piks_at_least_partially_correct = df_ops.persist(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(df_ops.drop_duplicates)
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id", "pik"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
piks_at_least_partially_correct

,record_id,pik,correct
npartitions=668,,,
,string,string,bool[pyarrow]
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [43]:
# Overall accuracy, treating it as a black box
piks_correct_proportion = (df_ops.compute(piks_at_least_partially_correct.correct.sum()) / piks_assigned)
piks_correct_proportion

0.9992742065347998

In [44]:
print(f'{piks_correct_proportion:.5%} of the PIKs assigned were correct; compare with {real_life_pvs_accuracy:.5%} in real life')

99.92742% of the PIKs assigned were correct; compare with 99.82079% in real life


In [45]:
# Looking at whether the exact *record* linked was from the same simulant
sim_record_links_at_least_partially_correct = df_ops.persist(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id_raw_input_file", "record_id_reference_file", "pik", "module_name", "pass_name"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
sim_record_links_at_least_partially_correct

,record_id_raw_input_file,record_id_reference_file,pik,module_name,pass_name,correct
npartitions=334,,,,,,
,string,string,string,string,string,bool[pyarrow]
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [46]:
len(sim_record_links_at_least_partially_correct)

303915660

In [47]:
len(df_ops.drop_duplicates(sim_record_links_at_least_partially_correct[['record_id_raw_input_file', 'record_id_reference_file']]))

303915660

In [48]:
(
    df_ops.compute(sim_record_links_at_least_partially_correct.correct.sum()) / piks_assigned
)

0.999271090538737

In [49]:
assert df_ops.compute((df_ops.groupby_agg_small_groups(confirmed_piks_with_ground_truth, by='record_id_raw_input_file', agg_func=lambda x: x.record_id_reference_file.nunique()) <= 1).all())

In [50]:
# Using definition 3 -- at the PIK level
piks_at_least_partially_correct = df_ops.persist(
    piks_at_least_partially_correct
        .rename(columns={'record_id': 'record_id_raw_input_file'})
        .merge(confirmed_piks_with_ground_truth[['record_id_raw_input_file', 'module_name', 'pass_name']], on='record_id_raw_input_file')
)
piks_at_least_partially_correct

,record_id_raw_input_file,pik,correct,module_name,pass_name
npartitions=668,,,,,
,string,string,bool[pyarrow],string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [51]:
# Accuracy by module -- note that this shows the opposite pattern (with the sample data)
# relative to the results of Layne et al., who found GeoSearch was much *more* accurate
df_ops.compute(piks_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
dobsearch,0.996523,154732
namesearch,0.997646,18325243
geosearch,0.999379,284705539
hhcompsearch,0.999899,730146


In [52]:
# Accuracy by pass -- could be used to tune pass-specific cutoffs, but
# this might not be too informative while we are still using the sample data.
df_ops.compute(piks_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                    
dobsearch    initials name switch                              0.996523   
namesearch   DOB and NYSIIS of name                            0.997645   
             DOB and initials                                   0.99899   
geosearch    geokey name switch                                0.999058   
             geokey                                            0.999272   
hhcompsearch year of birth                                     0.999738   
             initials                                          0.999965   
geosearch    house number and street name Soundex              0.999983   
             some name and DOB information                     0.999992   
             house number and street name Soundex name switch       1.0   

                                                                    size  
module_name  pass_name                                                    
dobsearch    initials name switch                                 154732  
namesearch   DOB and NYSIIS of name                             18323262  
             DOB and initials                                       1981  
geosearch    geokey name switch                                   876667  
             geokey                                            240966488  
hhcompsearch year of birth                                        213626  
             initials                                             516520  
geosearch    house number and street name Soundex               13077940  
             some name and DOB information                      29740242  
             house number and street name Soundex name switch      44202

In [53]:
# Using definition 3 -- at the link level
df_ops.compute(sim_record_links_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
dobsearch,0.996523,154732
namesearch,0.997646,18325243
geosearch,0.999376,284705539
hhcompsearch,0.999899,730146


In [54]:
df_ops.compute(sim_record_links_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                    
dobsearch    initials name switch                              0.996523   
namesearch   DOB and NYSIIS of name                            0.997645   
             DOB and initials                                   0.99899   
geosearch    geokey name switch                                 0.99905   
             geokey                                            0.999268   
hhcompsearch year of birth                                     0.999738   
             initials                                          0.999965   
geosearch    house number and street name Soundex              0.999983   
             some name and DOB information                     0.999992   
             house number and street name Soundex name switch       1.0   

                                                                    size  
module_name  pass_name                                                    
dobsearch    initials name switch                                 154732  
namesearch   DOB and NYSIIS of name                             18323262  
             DOB and initials                                       1981  
geosearch    geokey name switch                                   876667  
             geokey                                            240966488  
hhcompsearch year of birth                                        213626  
             initials                                             516520  
geosearch    house number and street name Soundex               13077940  
             some name and DOB information                      29740242  
             house number and street name Soundex name switch      44202

In [55]:
df_ops.compute(sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct].groupby(["module_name", "pass_name"]).size()).sort_values()

module_name   pass_name                           
namesearch    DOB and initials                             2
hhcompsearch  initials                                    18
              year of birth                               56
geosearch     house number and street name Soundex       218
              some name and DOB information              247
dobsearch     initials name switch                       538
geosearch     geokey name switch                         833
namesearch    DOB and NYSIIS of name                   43144
geosearch     geokey                                  176471
dtype: int64

### Incorrect and missed PIKs

In [56]:
incorrectly_linked_pairs = df_ops.persist(df_ops.drop_duplicates(
    sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct]
        [["record_id_raw_input_file", "record_id_reference_file"]]
))
incorrectly_linked_pairs

,record_id_raw_input_file,record_id_reference_file
npartitions=668,,
,string,string
,...,...
...,...,...
,...,...
,...,...


In [57]:
len(incorrectly_linked_pairs)

221527

In [58]:
incorrect_links = df_ops.head(incorrectly_linked_pairs, n=100)
incorrect_links

,record_id_raw_input_file,record_id_reference_file
0,simulated_census_2030_0_1000425,simulated_geobase_reference_file_284_1703768
1,simulated_census_2030_0_1000941,simulated_geobase_reference_file_25_467554
2,simulated_census_2030_0_1001091,simulated_geobase_reference_file_200_917055
3,simulated_census_2030_0_1001617,simulated_geobase_reference_file_191_1983837
4,simulated_census_2030_0_1001689,simulated_geobase_reference_file_247_460345
...,...,...
95,simulated_census_2030_0_185313,simulated_geobase_reference_file_156_1056267
96,simulated_census_2030_0_185604,simulated_geobase_reference_file_343_716434
97,simulated_census_2030_0_187817,simulated_geobase_reference_file_163_1501087
98,simulated_census_2030_0_18943,simulated_geobase_reference_file_16_1773322


In [ ]:
%xdel incorrectly_linked_pairs

In [61]:
comparison_cols = [
    "first_name",
    "middle_name",
    "last_name",
    "date_of_birth",
    "street_number",
    "street_name",
    "unit_number",
    "city",
    "state",
]

incorrect_links_detail = (
    incorrect_links
        .merge(
            census_2030_piked[census_2030_piked.record_id.isin(incorrect_links.record_id_raw_input_file)]
                .compute()
                .rename(columns={"record_id": "record_id_raw_input_file", "middle_initial": "middle_name"})
                [["record_id_raw_input_file"] + comparison_cols],
            on="record_id_raw_input_file",
            how="left",
        )
        .merge(
            reference_file[reference_file.record_id.isin(incorrect_links.record_id_reference_file)]
                .compute()
                .rename(columns={"record_id": "record_id_reference_file"})
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                [["record_id_reference_file"] + comparison_cols],
            on="record_id_reference_file",
            how="left",
            suffixes=("_census", "_reference_file"),
        )
)
def flatten(xss):
    return [x for xs in xss for x in xs]

incorrect_links_detail[flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])]

,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,Penny,<NA>,M,Solomon,Ezell,Ezell,09/15/2028,20280915,9639,9639,givhans rd,GIVHANS RD,<NA>,<NA>,parkville,PARKVILLE,MD,MD
1,Audrey,Jade,G,<NA>,Bell,Bell,03/06/2011,20110306,109,109,hubbell street,HUBBELL STREET,<NA>,<NA>,lorton,LORTON,VA,VA
2,<NA>,Amelia,O,Abby,Wilson,Wilson,10/23/2028,20281023,946,946,jessica lane,JESSICA LANE,<NA>,<NA>,monkton,MONKTON,MD,MD
3,Royalty,Camilla,E,Evelyn,Giordano,Giordano,10/09/2028,20281009,9136,9136,pontiac lake rd,PONTIAC LAKE RD,<NA>,<NA>,alameda,ALAMEDA,CA,CA
4,Alaia,Martin,A,Alexander,Reitz,Reitz,10/12/2028,20281012,6100,6100,burdette drive,BURDETTE DRIVE,<NA>,<NA>,chicago,CHICAGO,IL,IL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Kristin,Charles,E,<NA>,Bewick,Bewick,11/15/1963,19631115,20961,20961,gebron dr,GEBRON DR,<NA>,<NA>,salt lake city,SALT LAKE CITY,UT,UT
96,Brooklyn,David,E,Eric,Stange,Stange,02/19/2019,20190219,6539,6539,oahu isl,OAHU ISL,<NA>,<NA>,litl falls,LITL FALLS,NJ,NJ
97,Michael,Michelle,M,Marcia,Mcmahan,Mcmahan,04/06/1963,19630406,1114,1114,s lions spg wy,S LIONS SPG WY,<NA>,<NA>,austin,AUSTIN,TX,TX
98,<NA>,Elliott,C,Paul,Pacheco,Pacheco,11/29/2016,20161129,500,500,philadelphia way,PHILADELPHIA WAY,<NA>,<NA>,benton,BENTON,NY,NY


In [63]:
missed_links = df_ops.persist(
    census_2030_piked[census_2030_piked.pik.isnull()][["record_id"]]
        .merge(census_2030_ground_truth, on="record_id")
        .merge(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == 1], on="simulant_id", suffixes=("_census", "_reference_file"))
)

In [65]:
len(missed_links)

116621923

In [66]:
simulants_missed = df_ops.head(missed_links[['simulant_id']], n=100).simulant_id.unique()
simulants_missed

<ArrowStringArray>
[  '28_365249',   '28_623546',   '40_181957',   '40_188899',   '93_545215',
    '99_95719',   '99_880072',   '99_939969',  '103_100559', '107_1172955',
  '131_996214', '278_1021948', '315_1154921',  '323_191653',  '323_912938',
  '352_526740',   '446_78851', '465_1023090',   '478_82913',  '478_194997',
  '478_230323', '496_1177062',  '539_822093',  '539_953168',  '558_341841',
  '656_554266',  '656_776297',   '682_73548', '682_1103329',   '771_12558']
Length: 30, dtype: string

In [ ]:
missed_pairs = missed_links[missed_links.simulant_id.isin(list(simulants_missed))].compute()
missed_pairs

,record_id_census,simulant_id,possible_to_pik,record_id_reference_file,n_unique_simulants
0,simulated_census_2030_0_308282,28_365249,1.0,simulated_geobase_reference_file_380_744864,1
1,simulated_census_2030_0_308282,28_365249,1.0,simulated_name_dob_reference_file_115_755841,1
2,simulated_census_2030_0_308282,28_365249,1.0,simulated_geobase_reference_file_380_744865,1
3,simulated_census_2030_0_526134,28_623546,1.0,simulated_geobase_reference_file_119_1624848,1
4,simulated_census_2030_0_526134,28_623546,1.0,simulated_geobase_reference_file_119_1624849,1
...,...,...,...,...,...
95,simulated_census_2030_23_61933,682_73548,1.0,simulated_name_dob_reference_file_258_774653,1
96,simulated_census_2030_23_934916,682_1103329,1.0,simulated_geobase_reference_file_139_2223854,1
97,simulated_census_2030_23_934916,682_1103329,1.0,simulated_name_dob_reference_file_366_114787,1
98,simulated_census_2030_25_10574,771_12558,1.0,simulated_geobase_reference_file_232_595644,1


In [ ]:
%xdel missed_links

In [75]:
missed_links_detail = (
    missed_pairs
        .merge(census_2030_piked[census_2030_piked.record_id.isin(list(missed_pairs.record_id_census))].compute().rename(columns={"record_id": "record_id_census", "middle_initial": "middle_name"}), on="record_id_census")
        .merge(reference_file[reference_file.record_id.isin(missed_pairs.record_id_reference_file)].compute().rename(columns=lambda c: c.replace('mailing_address_', '')).rename(columns={"record_id": "record_id_reference_file"}), on="record_id_reference_file", suffixes=("_census", "_reference_file"))
)

In [76]:
for simulant in simulants_missed:
    print(simulant)
    display(missed_links_detail[missed_links_detail.simulant_id == simulant][['simulant_id'] + flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])])

28_365249


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,28_365249,Louis,Louis,R,Russell,Ansley,Ansley,10/07/2006,19681123,220,220,4th ave nw,4TH AVE NW,<NA>,<NA>,visalia,<NA>,KY,CA
1,28_365249,Louis,Louis,R,Russell,Ansley,Ansley,10/07/2006,19681123,220,<NA>,4th ave nw,<NA>,<NA>,<NA>,visalia,<NA>,KY,<NA>
2,28_365249,Louis,Louis,R,Russell,Ansley,Ansley,10/07/2006,19681123,220,220,4th ave nw,4TH AVE NW,<NA>,<NA>,visalia,VISALIA,KY,CA


28_623546


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
3,28_623546,Scott,Sceeter,J,Joseph,Justman,Justman,09/22/1975,19750922,6341,3,<NA>,CLEMENT RD,<NA>,<NA>,chandler,GRANDVIEW,AZ,MO
4,28_623546,Scott,Sceeter,J,Joseph,Justman,Justman,09/22/1975,19750922,6341,6341,<NA>,HUBBERT STREET,<NA>,<NA>,chandler,CHANDLER,AZ,AZ
5,28_623546,Scott,Sceeter,J,Joseph,Justman,Justman,09/22/1975,19750922,6341,<NA>,<NA>,<NA>,<NA>,<NA>,chandler,<NA>,AZ,<NA>


40_181957


...

,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
94,682_73548,Eric,Eric,T,Thomas,Villegas,Villegas,12/09/1998,19690309,65,65,parkwood drive,PARKWOOD DRIVE,<NA>,<NA>,frederick,FREDERICK,CO,CO
95,682_73548,Eric,Eric,T,Thomas,Villegas,Villegas,12/09/1998,19690309,65,<NA>,parkwood drive,<NA>,<NA>,<NA>,frederick,<NA>,CO,<NA>


682_1103329


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
96,682_1103329,Rachel,Rachel,T,T,Rodriguez,Rodriguez,11/03/1993,<NA>,441,5,belrose rd,ROYAL PALM BEACH BLVD,<NA>,<NA>,s diego,BETHLEHEM,CA,NY
97,682_1103329,Rachel,Rachel,T,T,Rodriguez,Rodriguez,11/03/1993,<NA>,441,<NA>,belrose rd,<NA>,<NA>,<NA>,s diego,<NA>,CA,<NA>


771_12558


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
98,771_12558,Sxott,Scott,T,Tyrone,Norris,Norris,04/30/1974,19740430,2215,2215,mountainside d,MOUNTAINSIDE D,<NA>,<NA>,amherst,AMHERST,NY,NY
99,771_12558,Sxott,Scott,T,Tyrone,Norris,Norris,04/30/1974,19740430,2215,<NA>,mountainside d,<NA>,<NA>,<NA>,amherst,<NA>,NY,<NA>
